#### Mục tiêu: Xây dựng mô hình có độ chính xác trên tập validation khoảng 94%

In [72]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

!wget --no-check-certificate \
    https://storage.googleapis.com/protonx-cloud-storage/bbc-text.csv \
    -O /tmp/bbc-text.csv

--2024-02-13 13:05:19--  https://storage.googleapis.com/protonx-cloud-storage/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.98.207, 142.250.97.207, 142.251.107.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.98.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [application/octet-stream]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.03s   

2024-02-13 13:05:19 (193 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [73]:
sentences = []
labels = []
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
print(len(stopwords))
# Expected Output
# 153

153


In [74]:
with open("/tmp/bbc-text.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        sentence = row[1]
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
        sentences.append(sentence)


print(len(labels))
print(len(sentences))
print(sentences[1])
# Expected Output
# 2225
# 2225
# tv future hands viewers home theatre systems  plasma high-definition tvs  digital video recorders moving living room  way people watch tv will radically different five years  time.  according expert panel gathered annual consumer electronics show las vegas discuss new technologies will impact one favourite pastimes. us leading trend  programmes content will delivered viewers via home networks  cable  satellite  telecoms companies  broadband service providers front rooms portable devices.  one talked-about technologies ces digital personal video recorders (dvr pvr). set-top boxes  like us s tivo uk s sky+ system  allow people record  store  play  pause forward wind tv programmes want.  essentially  technology allows much personalised tv. also built-in high-definition tv sets  big business japan us  slower take off europe lack high-definition programming. not can people forward wind adverts  can also forget abiding network channel schedules  putting together a-la-carte entertainment. us networks cable satellite companies worried means terms advertising revenues well  brand identity  viewer loyalty channels. although us leads technology moment  also concern raised europe  particularly growing uptake services like sky+.  happens today  will see nine months years  time uk   adam hume  bbc broadcast s futurologist told bbc news website. likes bbc  no issues lost advertising revenue yet. pressing issue moment commercial uk broadcasters  brand loyalty important everyone.  will talking content brands rather network brands   said tim hanlon  brand communications firm starcom mediavest.  reality broadband connections  anybody can producer content.  added:  challenge now hard promote programme much choice.   means  said stacey jolna  senior vice president tv guide tv group  way people find content want watch simplified tv viewers. means networks  us terms  channels take leaf google s book search engine future  instead scheduler help people find want watch. kind channel model might work younger ipod generation used taking control gadgets play them. might not suit everyone  panel recognised. older generations comfortable familiar schedules channel brands know getting. perhaps not want much choice put hands  mr hanlon suggested.  end  kids just diapers pushing buttons already - everything possible available   said mr hanlon.  ultimately  consumer will tell market want.   50 000 new gadgets technologies showcased ces  many enhancing tv-watching experience. high-definition tv sets everywhere many new models lcd (liquid crystal display) tvs launched dvr capability built  instead external boxes. one example launched show humax s 26-inch lcd tv 80-hour tivo dvr dvd recorder. one us s biggest satellite tv companies  directtv  even launched branded dvr show 100-hours recording capability  instant replay  search function. set can pause rewind tv 90 hours. microsoft chief bill gates announced pre-show keynote speech partnership tivo  called tivotogo  means people can play recorded programmes windows pcs mobile devices. reflect increasing trend freeing multimedia people can watch want  want.

2225
2225
worldcom boss  left books alone  former worldcom boss bernie ebbers  accused overseeing $11bn (£5.8bn) fraud  never made accounting decisions  witness told jurors.  david myers made comments questioning defence lawyers arguing mr ebbers not responsible worldcom s problems. phone company collapsed 2002 prosecutors claim losses hidden protect firm s shares. mr myers already pleaded guilty fraud assisting prosecutors.  monday  defence lawyer reid weingarten tried distance client allegations. cross examination  asked mr myers ever knew mr ebbers  make accounting decision  .  not aware   mr myers replied.  ever know mr ebbers make accounting entry worldcom books   mr weingarten pressed.  no   replied witness. mr myers admitted ordered false accounting entries request former worldcom chief financial officer scott sullivan. defence lawyers trying paint mr sullivan  admitted fraud will testify later trial  mastermind behind worldcom s accounting house cards.  mr ebbers  team  meanwhi

In [75]:
len(sentences[1].split())

200

In [76]:
vocab_size = len(sentences)
embedding_dim = 64
max_length = 200
# trunc_type = # FIXME
# padding_type = # FIXME
# oov_tok = # FIXME
training_portion = .8

In [77]:
data_size = len(sentences)

In [78]:
label_dict = {}
count = 0

for i in range(data_size):
    if labels[i] not in label_dict:
        label_dict[labels[i]] = count
        count += 1
    labels[i] = label_dict[labels[i]]

In [79]:
num_classes = len(label_dict)

In [80]:
train_size = int(0.8 * data_size)

train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

validation_sentences = sentences[train_size:]
validation_labels = labels[train_size:]

print(train_size)
print(len(train_sentences))
print(len(train_labels))
print(len(validation_sentences))
print(len(validation_labels))

# Expected output (if training_portion=.8)
# 1780
# 1780
# 1780
# 445
# 445

1780
1780
1780
445
445


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words = vocab_size, oov_token= '<OOV>')
tokenizer.fit_on_texts(train_sentences)
# word_index =

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded =  pad_sequences(train_sequences, maxlen = max_length,truncating = 'post', padding = 'post')
test_sequences = tokenizer.texts_to_sequences(validation_sentences)
test_padded = pad_sequences(test_sequences, maxlen = max_length,truncating = 'post', padding = 'post')

print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

# Expected Ouput
# 449
# 120
# 200
# 120
# 192
# 120

Exception ignored in: <function _xla_gc_callback at 0x7900b2535120>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


449
200
200
200
192
200


In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, RNN, Dense, LSTM, Bidirectional, SimpleRNN, GlobalAveragePooling1D
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
import tensorflow as tf

In [ ]:
model = Sequential([
  Embedding(vocab_size,embedding_dim, input_length = max_length),
  GlobalAveragePooling1D(),
  Dense(16, activation = 'relu'),
  Dense(num_classes, activation = 'softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(learning_rate=0.001),metrics=['accuracy'])
model.summary()

# Expected Output
# Layer (type)                 Output Shape              Param #
# =================================================================
# embedding (Embedding)        (None, 120, 16)           16000
# _________________________________________________________________
# global_average_pooling1d (Gl (None, 16)                0
# _________________________________________________________________
# dense (Dense)                (None, 24)                408
# _________________________________________________________________
# dense_1 (Dense)              (None, 6)                 150
# =================================================================
# Total params: 16,558
# Trainable params: 16,558
# Non-trainable params: 0

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 200, 64)           142400    
                                                                 
 global_average_pooling1d_7  (None, 64)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_16 (Dense)            (None, 16)                1040      
                                                                 
 dense_17 (Dense)            (None, 5)                 85        
                                                                 
Total params: 143525 (560.64 KB)
Trainable params: 143525 (560.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
train_padded[2]

array([   1,    1,    1,    1, 1105,   78,    4,    5,    1,  202,  451,
        908,    1,  169,  131,  434,  496,  925, 1377,    1,    1,    1,
       1501,  618,  871,   63,   48,  226,   59,   42,  607,    1,  573,
        303,    1,   29,   43,  199, 1105,  747,   63,  219,    1,  908,
          1,    5,  439, 1981,  909,    1,   44,   59,   44,  141,  141,
          1,    1,    1,    1,  363, 1459,    1,   93,  430,  379,  102,
        163,   39,  125, 1105,    1,  917,  414,  380,  434,  426, 1795,
       1393, 1956,    1,    1,  160,  485,  249,   33,  160,  373,  426,
          1,  626,  200,   56,  247, 2030,  439, 1460,  105,  578,   26,
        861,  496,  426,  618,  515,    3,    1,   66,   74,    1, 1279,
        619,   35,  949,   13,   26,   82,  434,  496, 1615,  446,    1,
          1,  253,   13,  835,    1,  182,    1,    4,    1,  242,    1,
       1834, 1704, 1106,   30, 1835,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [ ]:
num_epochs = 30
train_labels = np.array(train_labels)
validation_labels = np.array(validation_labels)
history = model.fit(train_padded, train_labels, epochs = num_epochs, validation_data=(test_padded,validation_labels))